In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import requests
import pickle
import os
import sys
sys.path.append(os.path.abspath("..")) 

from helpers import tankathon_bballgm_team_mapper
from data_retrieval.realgm_retr import get_realgm_allstar_rosters, get_realgm_allstar_roster

In [2]:
# constants
NYEARS = 5
DRAFT_HISTORY_ADJUSTMENTS_NYEARS = 5
ALL_STAR_ADJUSTMENT_NYEARS = 3

# Lottery Luck Adjustment

In [3]:
def get_draft_lottery_history(year_start, year_end, store_data: bool =True) -> dict[str, pd.DataFrame]:
    draft_lottery_results = {}
    years = list(range(year_start, year_end + 1))
    for year in years:

        # Extract the table for the current year
        url_format = "https://basketball.realgm.com/nba/draft/lottery_results/{year}"
        url = url_format.format(year=year)
        res = requests.get(url)
        soup = BeautifulSoup(res.text, 'html.parser')
        table = soup.find("table", class_="table-striped")

        if not table:
            print(f"No table found for year {year}")
            continue

        # convert the table to a DataFrame
        headers = [th.text.strip() for th in table.find_all("th")]
        rows = []
        for tr in table.find_all("tr")[1:]:  # Skip header
            row = [td.text.strip() for td in tr.find_all("td")]
            if row:
                rows.append(row)
        df = pd.DataFrame(rows, columns=headers)

        # Convert the "Pick Change" column to integers
        df["Pick Change"] = df["Pick Change"].str.replace("+", "", regex=False).astype(int)

        draft_lottery_results[year] = df

    if store_data:
        for year, df in draft_lottery_results.items():
            if not os.path.exists(f"data/draft_lottery_results_{year}.csv"):
                df.to_csv(f"data/draft_lottery_results_{year}.csv", index=False)
        # Save the results to a pickle file
        if not os.path.exists(f"data/draft_lottery_results_{year_start}_{year_end}.pkl"):
            with open(f"data/draft_lottery_results_{year_start}_{year_end}.pkl", "wb") as f:
                pickle.dump(draft_lottery_results, f)

    return draft_lottery_results

In [4]:
# collect draft lottery history data
max_year = datetime.now().year
min_year = max_year - DRAFT_HISTORY_ADJUSTMENTS_NYEARS
draft_lottery_history = get_draft_lottery_history(min_year, max_year - 1, store_data=True)

In [5]:
def get_current_lottery_odds() -> pd.DataFrame:

    url = 'https://www.tankathon.com/'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    table = soup.find("table", class_="draft-board")

    # convert the table to a DataFrame
    header_row = soup.find("tr", class_="headers")
    headers = [
        td.text.strip()
        for td in header_row.find_all("td")
        if 'mobile' not in td.get('class', [])
        and td.text.strip()  # Ignore empty cells
    ]
    rows = []
    for tr in table.find_all("tr", class_="pick-row-lottery"):
        tds = tr.find_all("td")
        row = []

        for i, td in enumerate(tds):
            # Special handling for team name column
            if "name" in td.get("class", []):
                desktop_team = td.find("div", class_="desktop")
                team_name = desktop_team.text.strip() if desktop_team else ""
                row.append(team_name)
            else:
                row.append(td.text.strip())

        if row:
            rows.append(row)
    return pd.DataFrame(rows, columns=headers)

In [6]:
# Load the current lottery odds
df_current = get_current_lottery_odds()
df_current['Team'] = df_current.Team.apply(lambda x: tankathon_bballgm_team_mapper[x])
df_current

,Pick,Team,Record,Win%,GB,Streak,L10,Top 4,#1 Ovr
0,110,Dallas Mavericks,39-43,.476,22.0,Lost 1,4-6,8.5%,1.8%
1,26,San Antonio Spurs,34-48,.415,17.0,Won 1,3-7,26.3%,6.0%
2,32,Philadelphia 76ers,24-58,.293,7.0,Lost 2,1-9,42.1%,10.5%
3,41,Charlotte Hornets,19-63,.232,2.0,Lost 7,1-9,52.1%,14.0%
4,54,Utah Jazz,17-65,.207,--,Lost 2,1-9,52.1%,14.0%
5,64,Washington Wizards,18-64,.220,1.0,Won 1,2-8,52.1%,14.0%
6,73,New Orleans Pelicans,21-61,.256,4.0,Lost 7,2-8,48.1%,12.5%
7,82,Brooklyn Nets,26-56,.317,9.0,Lost 3,3-7,37.2%,9.0%
8,92,Toronto Raptors,30-52,.366,13.0,Lost 2,5-5,31.9%,7.5%
9,101,Phoenix Suns,36-46,.439,19.0,Lost 1,1-9,17.3%,3.8%


In [7]:
# join lottery team previous lottery results
df_joined = df_current.copy()
for year in draft_lottery_history:
    pick_change_series = draft_lottery_history[year][["Team", "Pick Change"]].copy()
    pick_change_series.rename(columns={"Pick Change": f"Pick Change_{year}"}, inplace=True)
    df_joined = df_joined.join(pick_change_series.set_index("Team"), on="Team")
df_joined

,Pick,Team,Record,Win%,GB,Streak,L10,Top 4,#1 Ovr,Pick Change_2020,Pick Change_2021,Pick Change_2022,Pick Change_2023,Pick Change_2024
0,110,Dallas Mavericks,39-43,.476,22.0,Lost 1,4-6,8.5%,1.8%,NaN,NaN,NaN,0.0,NaN
1,26,San Antonio Spurs,34-48,.415,17.0,Won 1,3-7,26.3%,6.0%,0.0,0.0,0.0,2.0,1.0
2,32,Philadelphia 76ers,24-58,.293,7.0,Lost 2,1-9,42.1%,10.5%,NaN,NaN,NaN,NaN,NaN
3,41,Charlotte Hornets,19-63,.232,2.0,Lost 7,1-9,52.1%,14.0%,5.0,0.0,0.0,2.0,-3.0
4,54,Utah Jazz,17-65,.207,--,Lost 2,1-9,52.1%,14.0%,NaN,NaN,NaN,0.0,-2.0
5,64,Washington Wizards,18-64,.220,1.0,Won 1,2-8,52.1%,14.0%,0.0,NaN,0.0,0.0,0.0
6,73,New Orleans Pelicans,21-61,.256,4.0,Lost 7,2-8,48.1%,12.5%,0.0,0.0,NaN,0.0,NaN
7,82,Brooklyn Nets,26-56,.317,9.0,Lost 3,3-7,37.2%,9.0%,NaN,NaN,NaN,NaN,6.0
8,92,Toronto Raptors,30-52,.366,13.0,Lost 2,5-5,31.9%,7.5%,NaN,3.0,NaN,0.0,-2.0
9,101,Phoenix Suns,36-46,.439,19.0,Lost 1,1-9,17.3%,3.8%,0.0,NaN,NaN,NaN,NaN


In [8]:
def convert_odds_to_balls(df: pd.DataFrame) -> pd.DataFrame:
    # Convert the odds to integers
    df['start_balls'] = df['#1 Ovr'].replace({'%': ''}, regex=True).astype(float) * 10
    return df
df_joined = convert_odds_to_balls(df_joined)
df_joined

,Pick,Team,Record,Win%,GB,Streak,L10,Top 4,#1 Ovr,Pick Change_2020,Pick Change_2021,Pick Change_2022,Pick Change_2023,Pick Change_2024,start_balls
0,110,Dallas Mavericks,39-43,.476,22.0,Lost 1,4-6,8.5%,1.8%,NaN,NaN,NaN,0.0,NaN,18.0
1,26,San Antonio Spurs,34-48,.415,17.0,Won 1,3-7,26.3%,6.0%,0.0,0.0,0.0,2.0,1.0,60.0
2,32,Philadelphia 76ers,24-58,.293,7.0,Lost 2,1-9,42.1%,10.5%,NaN,NaN,NaN,NaN,NaN,105.0
3,41,Charlotte Hornets,19-63,.232,2.0,Lost 7,1-9,52.1%,14.0%,5.0,0.0,0.0,2.0,-3.0,140.0
4,54,Utah Jazz,17-65,.207,--,Lost 2,1-9,52.1%,14.0%,NaN,NaN,NaN,0.0,-2.0,140.0
5,64,Washington Wizards,18-64,.220,1.0,Won 1,2-8,52.1%,14.0%,0.0,NaN,0.0,0.0,0.0,140.0
6,73,New Orleans Pelicans,21-61,.256,4.0,Lost 7,2-8,48.1%,12.5%,0.0,0.0,NaN,0.0,NaN,125.0
7,82,Brooklyn Nets,26-56,.317,9.0,Lost 3,3-7,37.2%,9.0%,NaN,NaN,NaN,NaN,6.0,90.0
8,92,Toronto Raptors,30-52,.366,13.0,Lost 2,5-5,31.9%,7.5%,NaN,3.0,NaN,0.0,-2.0,75.0
9,101,Phoenix Suns,36-46,.439,19.0,Lost 1,1-9,17.3%,3.8%,0.0,NaN,NaN,NaN,NaN,38.0


In [9]:
# lottery luck adjustment
def adjust_lottery_luck(df: pd.DataFrame) -> pd.DataFrame:
    # Adjust the lottery luck based on the previous year's pick change
    df['balls_lottery_luck_adjusted'] = df['start_balls'].copy()
    pick_change_cols = [col for col in df.columns if 'Pick Change_' in col]
    for pick_change_col in pick_change_cols:
        year = pick_change_col.split('_')[-1]
        normalizer = datetime.now().year - int(year)
        df['balls_lottery_luck_adjusted'] -= (df[pick_change_col].fillna(0).astype(int)*10) / normalizer
    return df
df_adjusted = adjust_lottery_luck(df_joined.copy())
df_adjusted['#1 Ovr Adjusted'] = (df_adjusted['balls_lottery_luck_adjusted'] / df_adjusted['balls_lottery_luck_adjusted'].sum() * 100).round(2).astype(str) + '%'
df_adjusted.sort_values(by='balls_lottery_luck_adjusted', ascending=False, inplace=True)
df_adjusted

,Pick,Team,Record,Win%,GB,Streak,L10,Top 4,#1 Ovr,Pick Change_2020,Pick Change_2021,Pick Change_2022,Pick Change_2023,Pick Change_2024,start_balls,balls_lottery_luck_adjusted,#1 Ovr Adjusted
4,54,Utah Jazz,17-65,.207,--,Lost 2,1-9,52.1%,14.0%,NaN,NaN,NaN,0.0,-2.0,140.0,160.000000,18.1%
3,41,Charlotte Hornets,19-63,.232,2.0,Lost 7,1-9,52.1%,14.0%,5.0,0.0,0.0,2.0,-3.0,140.0,150.000000,16.97%
5,64,Washington Wizards,18-64,.220,1.0,Won 1,2-8,52.1%,14.0%,0.0,NaN,0.0,0.0,0.0,140.0,140.000000,15.84%
6,73,New Orleans Pelicans,21-61,.256,4.0,Lost 7,2-8,48.1%,12.5%,0.0,0.0,NaN,0.0,NaN,125.0,125.000000,14.14%
2,32,Philadelphia 76ers,24-58,.293,7.0,Lost 2,1-9,42.1%,10.5%,NaN,NaN,NaN,NaN,NaN,105.0,105.000000,11.88%
8,92,Toronto Raptors,30-52,.366,13.0,Lost 2,5-5,31.9%,7.5%,NaN,3.0,NaN,0.0,-2.0,75.0,87.500000,9.9%
10,111,Portland Trail Blazers,36-46,.439,19.0,Won 1,4-6,16.9%,3.7%,NaN,NaN,-1.0,2.0,-3.0,37.0,60.333333,6.83%
1,26,San Antonio Spurs,34-48,.415,17.0,Won 1,3-7,26.3%,6.0%,0.0,0.0,0.0,2.0,1.0,60.0,40.000000,4.53%
9,101,Phoenix Suns,36-46,.439,19.0,Lost 1,1-9,17.3%,3.8%,0.0,NaN,NaN,NaN,NaN,38.0,38.000000,4.3%
7,82,Brooklyn Nets,26-56,.317,9.0,Lost 3,3-7,37.2%,9.0%,NaN,NaN,NaN,NaN,6.0,90.0,30.000000,3.39%


# All Star Rest Penalty Adjustment

In [10]:
max_year = datetime.now().year
min_year = max_year - ALL_STAR_ADJUSTMENT_NYEARS
years = list(range(min_year, max_year))
as_df = get_realgm_allstar_rosters(years)
as_players = as_df.Player.values.tolist()

In [11]:
from data_retrieval.games_list_requester import GamesListRequester

Requester = GamesListRequester()
Requester.get_games_list(season_id=f'{max_year-1}-{str(max_year)[-2:]}', season_type='Regular Season')
games_df = Requester.df.copy()

# add team id to df_adjusted
if 'Team_Id' not in df_adjusted.columns:
    df_adjusted = df_adjusted.merge(games_df[['TEAM_NAME','TEAM_ID']].drop_duplicates(subset='TEAM_NAME', keep='first'), how='left', left_on='Team', right_on='TEAM_NAME')
    df_adjusted.drop(columns=['TEAM_NAME'], inplace=True)
    df_adjusted.rename(columns={'TEAM_ID': 'Team_Id'}, inplace=True)

# drop games from games_df that don't include teams in df_adjusted
games_df = games_df[games_df['TEAM_ID'].isin(df_adjusted['Team_Id'].unique())]
games_df.drop_duplicates(subset=['GAME_ID'], inplace=True) # remove duplicate games
games_df.reset_index(drop=True, inplace=True)

In [ ]:
# from io import StringIO
# from bs4 import BeautifulSoup
# import pandas as pd

import requests
from IPython.display import clear_output

url_format = "https://cdn.nba.com/static/json/liveData/boxscore/boxscore_{game_id}.json"
games_played_df_list = []
for i, row in games_df.iterrows():
    vs_str = row.MATCHUP.lower().replace(' ', '-').replace('.', '')
    game_id = row.GAME_ID
    url = url_format.format(game_id=game_id)

    response = requests.get(url)
    data = response.json()

    for team_type in ['home', 'away']:
        team = data['game'][f'{team_type}Team']
        player_data = [
            {
                "personId": p["personId"],
                "played": int(p.get("played", 0)),  # cast to int just in case
                "name": p["name"],
                "teamId": team['teamId'],
                'teamAbbr': team['teamTricode'],
            }
            for p in team['players']
        ]
        player_df = pd.DataFrame(player_data)
        games_played_df_list.append(player_df)
    
    clear_output(wait=True)
    print(f"Processed game {i+1}/{len(games_df)}: {game_id}")

games_played_df = pd.concat(games_played_df_list)
        # if team['teamId'] == row.TEAM_ID:
        #     for player in team['players']:
        #         if player['name'] in as_players:
        #             if int(player['played']) == 0:
        #                 # missed games
        #                 df_adjusted.loc[df_adjusted.Team_Id == team['teamId'], 'missed_games'] += 1

Processed game 894/894: 0022400066


In [12]:
# storing in df and dumping intermediate data to files for reusability

# read pkl for games_played_df if games_played_df_list does not exist
if 'games_played_df_list' not in locals():
    try:
        with open(f'data/games_played_{max_year-1}-{str(max_year)[-2:]}.pkl', "rb") as f:
            games_played_df = pickle.load(f)
    except FileNotFoundError:
        games_played_df = pd.DataFrame()

else:
    games_played_df = pd.concat(games_played_df_list)
    fbase = f'games_played_{max_year-1}-{str(max_year)[-2:]}'
    if not os.path.exists(f'data/{fbase}.csv'):
        games_played_df.to_csv(f'data/{fbase}.csv', index=False)
    if not os.path.exists(f'data/{fbase}.pkl'):
        with open(f'data/{fbase}.pkl', "wb") as f:
            pickle.dump(games_played_df, f)

In [13]:
# counting missed games by all stars for each team

df_adjusted['missed_games'] = 0
for i, player_data in games_played_df.iterrows():
    logic = (player_data['teamId'] in df_adjusted.Team_Id.values) & \
            (player_data['name'] in as_players) & \
            (int(player_data['played']) == 0)
    if logic:
        df_adjusted.loc[df_adjusted.Team_Id == player_data['teamId'], 'missed_games'] += 1

In [14]:
import math
# subtract 0.5 lottery balls per missed game by all star player
df_adjusted['balls_lottery_luck_adjusted'] -= df_adjusted['missed_games'].apply(lambda x: math.ceil(x * 0.5))
df_adjusted['#1 Ovr Adjusted'] = (df_adjusted['balls_lottery_luck_adjusted'] / df_adjusted['balls_lottery_luck_adjusted'].sum() * 100).round(2).astype(str) + '%'

In [15]:
df_adjusted

,Pick,Team,Record,Win%,GB,Streak,L10,Top 4,#1 Ovr,Pick Change_2020,Pick Change_2021,Pick Change_2022,Pick Change_2023,Pick Change_2024,start_balls,balls_lottery_luck_adjusted,#1 Ovr Adjusted,Team_Id,missed_games
0,54,Utah Jazz,17-65,.207,--,Lost 2,1-9,52.1%,14.0%,NaN,NaN,NaN,0.0,-2.0,140.0,142.000000,22.16%,1610612762,35
1,41,Charlotte Hornets,19-63,.232,2.0,Lost 7,1-9,52.1%,14.0%,5.0,0.0,0.0,2.0,-3.0,140.0,132.000000,20.6%,1610612766,35
2,64,Washington Wizards,18-64,.220,1.0,Won 1,2-8,52.1%,14.0%,0.0,NaN,0.0,0.0,0.0,140.0,131.000000,20.44%,1610612764,18
3,73,New Orleans Pelicans,21-61,.256,4.0,Lost 7,2-8,48.1%,12.5%,0.0,0.0,NaN,0.0,NaN,125.0,73.000000,11.39%,1610612740,103
4,32,Philadelphia 76ers,24-58,.293,7.0,Lost 2,1-9,42.1%,10.5%,NaN,NaN,NaN,NaN,NaN,105.0,38.000000,5.93%,1610612755,134
5,92,Toronto Raptors,30-52,.366,13.0,Lost 2,5-5,31.9%,7.5%,NaN,3.0,NaN,0.0,-2.0,75.0,78.500000,12.25%,1610612761,17
6,111,Portland Trail Blazers,36-46,.439,19.0,Won 1,4-6,16.9%,3.7%,NaN,NaN,-1.0,2.0,-3.0,37.0,60.333333,9.41%,1610612757,0
7,26,San Antonio Spurs,34-48,.415,17.0,Won 1,3-7,26.3%,6.0%,0.0,0.0,0.0,2.0,1.0,60.0,30.000000,4.68%,1610612759,19
8,101,Phoenix Suns,36-46,.439,19.0,Lost 1,1-9,17.3%,3.8%,0.0,NaN,NaN,NaN,NaN,38.0,24.000000,3.75%,1610612756,27
9,82,Brooklyn Nets,26-56,.317,9.0,Lost 3,3-7,37.2%,9.0%,NaN,NaN,NaN,NaN,6.0,90.0,30.000000,4.68%,1610612751,0


In [16]:
test = df_adjusted[['Team','balls_lottery_luck_adjusted','#1 Ovr Adjusted']].copy()

## Top 3 Pick Recency Penalty

In [195]:
# 1/4 scaled by years since for recent number 1 pick
# 1/2 scaled by years since for recent number 2 pick
# 3/4 scaled by years since for recent number 3 pick
test = df_adjusted.copy()
for year, draft_dict in draft_lottery_history.items():
    pick_series = draft_dict[['Team','Pick']]

    for pick, adjustment in enumerate([0.75, 0.5, 0.25]):
        team_name = pick_series.iloc[pick]['Team']
        # team_name_mapped = tankathon_bballgm_team_mapper.get(team_name, team_name)
        if test[test['Team'] == team_name].empty:
            continue

        test.loc[test['Team'] == team_name, 'balls_lottery_luck_adjusted'] *= (1 - (adjustment / (int(max_year) - int(year))))

    # round up balls_lottery_luck_adjusted
    test.loc[:,'balls_lottery_luck_adjusted'] = test['balls_lottery_luck_adjusted'].apply(lambda x: math.ceil(x))

test['#1 Ovr Adjusted'] = (test['balls_lottery_luck_adjusted'] / test['balls_lottery_luck_adjusted'].sum() * 100).round(2).astype(str) + '%'

In [196]:
test

,Pick,Team,Record,Win%,GB,Streak,L10,Top 4,#1 Ovr,Pick Change_2020,Pick Change_2021,Pick Change_2022,Pick Change_2023,Pick Change_2024,start_balls,balls_lottery_luck_adjusted,#1 Ovr Adjusted,Team_Id,missed_games
0,54,Utah Jazz,17-65,.207,--,Lost 2,1-9,52.1%,14.0%,NaN,NaN,NaN,0.0,-2.0,140.0,142.0,24.61%,1610612762,35
1,41,Charlotte Hornets,19-63,.232,2.0,Lost 7,1-9,52.1%,14.0%,5.0,0.0,0.0,2.0,-3.0,140.0,95.0,16.46%,1610612766,35
2,64,Washington Wizards,18-64,.220,1.0,Won 1,2-8,52.1%,14.0%,0.0,NaN,0.0,0.0,0.0,140.0,66.0,11.44%,1610612764,18
3,73,New Orleans Pelicans,21-61,.256,4.0,Lost 7,2-8,48.1%,12.5%,0.0,0.0,NaN,0.0,NaN,125.0,73.0,12.65%,1610612740,103
4,32,Philadelphia 76ers,24-58,.293,7.0,Lost 2,1-9,42.1%,10.5%,NaN,NaN,NaN,NaN,NaN,105.0,38.0,6.59%,1610612755,134
5,92,Toronto Raptors,30-52,.366,13.0,Lost 2,5-5,31.9%,7.5%,NaN,3.0,NaN,0.0,-2.0,75.0,79.0,13.69%,1610612761,17
6,111,Portland Trail Blazers,36-46,.439,19.0,Won 1,4-6,16.9%,3.7%,NaN,NaN,-1.0,2.0,-3.0,37.0,54.0,9.36%,1610612757,0
7,26,San Antonio Spurs,34-48,.415,17.0,Won 1,3-7,26.3%,6.0%,0.0,0.0,0.0,2.0,1.0,60.0,19.0,3.29%,1610612759,19
8,101,Phoenix Suns,36-46,.439,19.0,Lost 1,1-9,17.3%,3.8%,0.0,NaN,NaN,NaN,NaN,38.0,24.0,4.16%,1610612756,27
9,82,Brooklyn Nets,26-56,.317,9.0,Lost 3,3-7,37.2%,9.0%,NaN,NaN,NaN,NaN,6.0,90.0,23.0,3.99%,1610612751,0


In [183]:
test.loc[test['Team'] == team_name, 'balls_lottery_luck_adjusted']

9    22.5
Name: balls_lottery_luck_adjusted, dtype: float64

In [181]:
max_year

2025

In [182]:
(1 - (adjustment / (int(max_year) - int(year))))

0.75

In [199]:
df_display = test[['Team','#1 Ovr Adjusted', 'balls_lottery_luck_adjusted']]
df_display

,Team,#1 Ovr Adjusted,balls_lottery_luck_adjusted
0,Utah Jazz,24.61%,142.0
1,Charlotte Hornets,16.46%,95.0
2,Washington Wizards,11.44%,66.0
3,New Orleans Pelicans,12.65%,73.0
4,Philadelphia 76ers,6.59%,38.0
5,Toronto Raptors,13.69%,79.0
6,Portland Trail Blazers,9.36%,54.0
7,San Antonio Spurs,3.29%,19.0
8,Phoenix Suns,4.16%,24.0
9,Brooklyn Nets,3.99%,23.0


In [153]:
draft_lottery_history

{2020:    Pick                    Team Record    Odds Chances Pre-Lottery Position  \
 0     1  Minnesota Timberwolves  19-45  14.00%     140                  3rd   
 1     2   Golden State Warriors  15-50  14.00%     140                  1st   
 2     3       Charlotte Hornets  23-42   6.00%      60                  8th   
 3     4           Chicago Bulls  22-43   7.50%      75                  7th   
 4     5     Cleveland Cavaliers  19-46  14.00%     140                  2nd   
 5     6           Atlanta Hawks  20-47  12.50%     125                  4th   
 6     7         Detroit Pistons  20-46  10.50%     105                  5th   
 7     8         New York Knicks  21-45   9.00%      90                  6th   
 8     9      Washington Wizards  25-47   4.50%      45                  9th   
 9    10            Phoenix Suns  34-39   3.00%      30                 10th   
 10   11       San Antonio Spurs  32-39   2.00%      20                 11th   
 11   12        Sacramento Kings  